In [ ]:
df_numeric = data2.select_dtypes(include=['float64', 'int64'])

# Standardize the data with z-score
df_numeric = df_numeric.apply(zscore)

# Determine the number of rows and columns for the subplot grid
n = len(df_numeric.columns)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*3))

# Flatten the 2D array to 1D array of axes
axes = axes.ravel()

for i, column in enumerate(df_numeric):
    df_numeric.boxplot(column, ax=axes[i])

# Remove empty subplots
if n < nrows*ncols:
    for j in range(i+1, nrows*ncols):
        fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:

# Standardize the data with z-score
df_numeric = df_numeric.apply(zscore)

# Determine the number of rows and columns for the subplot grid
n = len(df_numeric.columns)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*3))

# Flatten the 2D array to 1D array of axes
axes = axes.ravel()

for i, column in enumerate(df_numeric):
    df_numeric.boxplot(column, ax=axes[i])

# Remove empty subplots
if n < nrows*ncols:
    for j in range(i+1, nrows*ncols):
        fig.delaxes(axes[j])

plt.tight_layout()
plt.show()